In [ ]:
import os, sys
current_dir = os.path.dirname(os.getcwd())
up_dir = os.path.dirname(current_dir)
sys.path.append(current_dir)
sys.path.append(up_dir)
from common_imports import *

In [ ]:
pipe_1_5 = AutoencoderKL.from_pretrained("AITSIS/imagine-v1", subfolder="vae", torch_type=torch.float16)
pipe_1_5.to("cuda:0", torch.float16)

In [ ]:
pipe_xl = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",  torch_type=torch.float16)
pipe_xl.to("cuda:1", torch.float16)

In [ ]:
pipe_1_5_base = AutoencoderKL.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="vae", torch_type=torch.float16)
pipe_1_5_base.to("cuda:2", torch.float16)

In [ ]:
image = Image.open("../../media/training_images/100_lora paisey_1024/19-5539.png").convert("RGB")
transformed_image = torchvision.transforms.ToTensor()(image).unsqueeze(0) * 2 - 1
#transformed_image = transformed_image.to("cuda:1", torch.float16)
print(transformed_image.shape)
print(transformed_image.dtype)

In [ ]:
outputs  = []
output_1_5 = pipe_1_5(transformed_image.to("cuda:0", torch.float16))
output_xl = pipe_xl(transformed_image.to("cuda:1", torch.float16))
output_1_5_base = pipe_1_5_base(transformed_image.to("cuda:2", torch.float16))
outputs.append({"imagine": output_1_5.sample,
                "output_xl": output_xl.sample,
                "output_1_5_base": output_1_5_base.sample})



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for i, output in enumerate(outputs):
    for key, value in output.items():
        out_image = (value / 2 + 0.5).clamp(0,1).squeeze(0)
        out_image =  out_image.detach().cpu().permute(1,2,0).numpy()  
        out_image = (out_image * 255).round().astype("uint8") 
        out_pil = Image.fromarray(out_image)

        out_pil.save(f"../../media/outputs/{key}.png")
        